# First part

This code runs the game but asks for input from a player (you can play the game)

Please note that in Jupyter notebook after a few steps it stops showing the progress and it will only shows the current state

In [1]:
import gym_2048
import gym
import numpy as np

if __name__ == '__main__':
  env = gym.make('2048-v0')
  env.seed()
  obs = env.reset()
  env.render()
  buffer = []
  done = False
  moves = 0
  count_action = 0
  old_action = 0
  while not done:
    action = int(input('Next action: '))
    if old_action == action:
      count_action +=1
    
    if count_action > 10:
      action = env.np_random.choice(range(4), 1).item()
      count_action = 0

    next_state, reward, done, info = env.step(action)
    moves += 1
    transition = (obs, action, reward, done, next_state)
    buffer.append(transition)
    obs = next_state
    old_action = action
    

    print('Next Action: "{}"\n\nReward: {}'.format(
      gym_2048.Base2048Env.ACTION_STRING[action], reward))
    env.render()
  with open('game.npy', 'wb') as f:
      np.save(f, buffer)

  print('\nTotal Moves: {}'.format(moves))

c:\Users\Celso\anaconda3\envs\Test\lib\site-packages\gym_2048\env.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  board[tile_locs] = tiles


4 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "right"

Reward: 0
0 	0 	0 	4
2 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "right"

Reward: 0
0 	0 	0 	4
0 	0 	0 	2
0 	0 	2 	0
0 	0 	0 	0
Next Action: "right"

Reward: 0
0 	0 	0 	4
0 	0 	0 	2
0 	0 	0 	2
0 	0 	2 	0
Next Action: "up"

Reward: 4
0 	0 	2 	4
0 	0 	0 	4
0 	0 	0 	0
0 	0 	0 	2
Next Action: "up"

Reward: 8
0 	2 	2 	8
0 	0 	0 	2
0 	0 	0 	0
0 	0 	0 	0
Next Action: "right"

Reward: 4
0 	2 	4 	8
0 	0 	0 	2
0 	0 	0 	0
0 	0 	0 	0
Next Action: "down"

Reward: 0
0 	0 	0 	0
2 	0 	0 	0
0 	0 	0 	8
0 	2 	4 	2
Next Action: "up"

Reward: 0
2 	2 	4 	8
0 	0 	0 	2
0 	0 	0 	0
0 	0 	0 	2
Next Action: "up"

Reward: 4
2 	2 	4 	8
2 	0 	0 	4
0 	0 	0 	0
0 	0 	0 	0
Next Action: "down"

Reward: 4
0 	0 	0 	0
0 	2 	0 	0
0 	0 	0 	8
4 	2 	4 	4
Next Action: "right"

Reward: 8
0 	0 	0 	0
0 	0 	2 	2
0 	0 	0 	8
0 	4 	2 	8
Next Action: "down"

Reward: 20
0 	0 	2 	0
0 	0 	0 	0
0 	0 	0 	2
0 	4 	4 	16
Next Action: "right"

Reward: 8
0 	0 	0 	2
0 	0 	0 	0
0 	

c:\Users\Celso\anaconda3\envs\Test\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Second part

Runs one example of the code, after it is done, it saves a Pickle file that can be used into `show_played_game.py` in a similar way to a real game

In [69]:
import gym_2048
import gym
import torch
from dqn_model import DQN
from new_action import make_action
import pickle

if __name__ == '__main__':
  env = gym.make('2048-v0')
  env.seed()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  online_net = DQN(size_state=4, num_actions=4)
  online_net.load_state_dict(torch.load('./DDQN_model.pth'))
  online_net.eval()
  obs = env.reset()
  env.render()

  done = False
  moves = 0
  count_action = 0
  old_action = 0
  total_reward = 0
  action_buffer = []
  boards = []
  score = []
  best_episode = {}

  while not done:
    action = make_action(online_net, obs, device)
    if old_action == action:
      count_action +=1
    
    if count_action > 10:
      action = env.np_random.choice(range(4), 1).item()
      count_action = 0

    next_state, reward, done, info = env.step(action)

    action_buffer.append(action)
    boards.append(obs)
    score.append(reward)
    moves += 1
    obs = next_state
    old_action = action
    total_reward += reward

    print('Next Action: "{}"\n\nReward: {}'.format(
      gym_2048.Base2048Env.ACTION_STRING[action], reward))
    env.render()

  print('\nTotal Moves: {}'.format(moves))
  best_episode["actions"] = action_buffer
  best_episode["boards"] = boards
  best_episode["score"] = score
  with open("./Render_game/best_episode.pkl", "wb") as pickle_file:
    pickle.dump(best_episode, pickle_file, pickle.HIGHEST_PROTOCOL)


c:\Users\Celso\anaconda3\envs\Test\lib\site-packages\gym_2048\env.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  board[tile_locs] = tiles


0 	0 	2 	0
0 	0 	0 	0
0 	0 	0 	0
2 	0 	0 	0
Next Action: "up"

Reward: 0
2 	0 	2 	0
0 	0 	0 	2
0 	0 	0 	0
0 	0 	0 	0
Next Action: "left"

Reward: 4
4 	2 	0 	0
2 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "right"

Reward: 0
0 	0 	4 	2
0 	0 	0 	2
0 	0 	0 	0
2 	0 	0 	0
Next Action: "left"

Reward: 0
4 	2 	0 	0
2 	0 	0 	0
0 	0 	0 	0
2 	0 	2 	0
Next Action: "up"

Reward: 4
4 	2 	2 	0
4 	0 	0 	0
0 	0 	0 	2
0 	0 	0 	0
Next Action: "up"

Reward: 8
8 	2 	2 	2
2 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "left"

Reward: 4
8 	4 	2 	0
2 	0 	0 	0
0 	0 	2 	0
0 	0 	0 	0
Next Action: "up"

Reward: 4
8 	4 	4 	0
2 	0 	2 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "left"

Reward: 12
8 	8 	0 	2
4 	0 	0 	0
0 	0 	0 	0
0 	0 	0 	0
Next Action: "right"

Reward: 16
0 	0 	16 	2
0 	0 	0 	4
0 	0 	0 	2
0 	0 	0 	0
Next Action: "left"

Reward: 0
16 	2 	0 	4
4 	0 	0 	0
2 	0 	0 	0
0 	0 	0 	0
Next Action: "left"

Reward: 0
16 	2 	4 	0
4 	0 	2 	0
2 	0 	0 	0
0 	0 	0 	0
Next Action: "up"

Reward: 0
16 	2 	4 	0
4 	0 	2 	0
2 	0 

In [68]:
print(total_reward)
env.render()

1196
8 	2 	32 	8
2 	16 	8 	16
16 	128 	32 	8
2 	8 	4 	2
